In [1]:
### Need to create a .py file which can add new data to database
## should check if data exists
## should allow data to 

In [2]:
import pandas as pd
import sys
sys.path.append("../source")

import df_db as df_db

In [3]:
trips = pd.read_csv("../data/E-Scooter_Trips_-_2019_Pilot.csv")
trips = trips.dropna(subset = ['Start Centroid Latitude', 'End Centroid Latitude'])

In [4]:
origin = trips.groupby(['Start Community Area Number','Start Community Area Name'])  \
        .size().reset_index().rename(columns={0:'count',
                                              'Start Community Area Number':'id',
                                              'Start Community Area Name':'community'
                                             }
                                    )
dest   = trips.groupby(['End Community Area Number','End Community Area Name'])  \
        .size().reset_index().rename(columns={0:'count',
                                              'End Community Area Number':'id',
                                              'End Community Area Name':'community'
                                             }
                                    )
communities = pd.concat([origin, dest])[['id','community']].drop_duplicates().sort_values(by = 'id')


In [5]:
trips.columns = trips.columns.str.replace(" ", "_")

In [6]:
db_conn = df_db.db_df_loader('micro_mobility')

Connecting to the PostgreSQL database...
Connection successful


In [7]:
db_conn.pd_to_db(trips,'trips')

copy_from_stringio() done


In [8]:
db_conn.pd_to_db(communities,'communities')

copy_from_stringio() done


In [9]:
%reload_ext sql
%sql postgresql://localhost/micro_mobility

In [10]:
%sql \dt

 * postgresql://localhost/micro_mobility
2 rows affected.


Schema,Name,Type,Owner
public,communities,table,networkadministrator
public,trips,table,networkadministrator


In [12]:
%sql \d+ communities

 * postgresql://localhost/micro_mobility
3 rows affected.


Column,Type,Modifiers,Storage,Stats target,Description
index,bigint,,plain,None,None
id,double precision,,plain,None,None
community,text,,extended,None,None
